# 01 Jul 2020 - Read results from MKMCXX microkinetics simulations

- This notebook produces DRC and TOF plots based on microkinetics data for the PtRu alloy project.
- Want to quickly read in the microkinetics simulation results from MKMCXX code.
- Basically, I used my `mkmcxx` class to automate reading the results of each simulation and extract the parameters required to produce a TOF volcano plot and degree of rate control plots.

In [ ]:
import os
import glob
import pickle
import re
import copy
import datetime

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from samueldy_atomistic_utils.microkinetics import mkmcxx

import tqdm
import multiprocessing

In [ ]:
# Other resources
%load_ext blackcellmagic

# Autoreload functionality for iterating on packages.
# Uncomment if running cells gets too slow.
%load_ext autoreload
%autoreload 2

Other utility functions:

In [ ]:
def load_from_file(fname, module=pickle):
    """
    Quick function to load an object from a file.

    By default, use the pickle module.
    """

    with open(fname, "rb") as f:
        obj = module.load(f)

    return obj

def save_to_file(obj, fname, module=pickle):
    """
    Quick function to save a single object to a file.

    By default, use the pickle module.
    """

    with open(fname, "wb") as f:
        module.dump(obj, f)

    return True

    # try:

    # except:
    #     print("Something went wrong with the pickling.")

In [ ]:
# Thanks to https://scentellegher.github.io/visualization/2018/05/02/custom-fonts-matplotlib.html
# specify the custom font to use
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "Nimbus Sans,Arial"
plt.rcParams["mathtext.fontset"] = "custom"
plt.rcParams["font.size"] = 22
plt.rcParams["axes.linewidth"] = 3
plt.rcParams["axes.titlepad"] = 15
plt.rcParams["text.usetex"] = True

# Customize ticks
plt.rcParams["xtick.bottom"] = True
plt.rcParams["xtick.top"] = True
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.left"] = True
plt.rcParams["ytick.right"] = True
plt.rcParams["ytick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 6
plt.rcParams["xtick.major.width"] = 3
plt.rcParams["ytick.major.size"] = 6
plt.rcParams["ytick.major.width"] = 3

In [ ]:
# Custom LaTeX preamble to use when rendering
plt.rcParams["text.usetex"] = True
preamble=r"""
\usepackage{amsmath}
\usepackage{amssymb}

% Use Helvetica for text and math
% Thanks to https://gist.github.com/Pni0/2923266
\renewcommand{\familydefault}{\sfdefault}
\usepackage[scaled=1]{helvet}
\usepackage[helvet]{sfmath}
\everymath={\sf}

% Shorten reaction arrows
\usepackage[version=4, arrows=pgf]{mhchem}

% Make arrows in chemical formulae shorter
\ExplSyntaxOn{}
\keys_define:nn { mhchem }
 {
  arrow-min-length .code:n =
   \cs_set:Npn \__mhchem_arrow_options_minLength:n { {#1} } % default is 2em
 }
\ExplSyntaxOff{}
\mhchemoptions{arrow-min-length=1.2em}
"""
plt.rcParams["text.latex.preamble"] = preamble

In [ ]:
# Enumerate all compounds and reaction strings used
compound_strings = [
    "Haqu",
    "NO3-",
    "H2",
    "N2",
    "N2O",
    "H2O",
    "NO",
    "NH3",
    "NO3*",
    "H*",
    "NO2*",
    "NO*",
    "N*",
    "NH*",
    "NH2*",
    "NH3*",
    "O*",
    "N2*",
    "N2O*",
    "OH*",
    "H2O*",
    "*",
]

rxn_strings = [
    "NO + * <-> NO*",
    "N2 + * <-> N2*",
    "N2O + * <-> N2O*",
    "H2O + * <-> H2O*",
    "NH3 + * <-> NH3*",
    "NO3- + * <-> NO3*",
    "NO3* + * <-> NO2* + O*",
    "NO2* + * <-> NO* + O*",
    "NO* + * <-> N* + O*",
    "2 N* <-> N2* + *",
    "2 NO* <-> N2O* + O*",
    "N2O* + * <-> N2* + O*",
    "N* + Haqu <-> NH*",
    "NH* + Haqu <-> NH2*",
    "NH2* + Haqu <-> NH3*",
    "O* + Haqu <-> OH*",
    "OH* + Haqu <-> H2O*",
    "Haqu + * <-> H*",
    "H2 + 2 * <-> 2 H*",
]

rxn_labels_latex = {
    "NO + * <-> NO*"        : r"$\ce{NO + {}^* \rightleftharpoons NO^*}$",
    "N2 + * <-> N2*"        : r"$\ce{N2 + {}^* \rightleftharpoons N2^*}$",
    "N2O + * <-> N2O*"      : r"$\ce{N2O + {}^* \rightleftharpoons N2O^*}$",
    "H2O + * <-> H2O*"      : r"$\ce{H2O + {}^* \rightleftharpoons H2O^*}$",
    "NH3 + * <-> NH3*"      : r"$\ce{NH3 + {}^* \rightleftharpoons NH3^*}$",
    "NO3- + * <-> NO3*"     : r"$\ce{NO3- + {}^* \rightleftharpoons NO3^*}$",
    "NO3* + * <-> NO2* + O*": r"$\ce{NO3^* + {}^* \rightleftharpoons NO2^* + O^*}$",
    "NO2* + * <-> NO* + O*" : r"$\ce{NO2^* + {}^* \rightleftharpoons NO^* + O^*}$",
    "NO* + * <-> N* + O*"   : r"$\ce{NO^* + {}^* \rightleftharpoons N^* + O^*}$",
    "2 N* <-> N2* + *"      : r"$\ce{2N^* \rightleftharpoons N2^* + {}^* }$",
    "2 NO* <-> N2O* + O*"   : r"$\ce{2NO^* \rightleftharpoons N2O^* + O^*}$",
    "N2O* + * <-> N2* + O*" : r"$\ce{N2O^* + {}^* \rightleftharpoons N2^* + O^*}$",
    "N* + Haqu <-> NH*"     : r"$\ce{N^* + H+ + e- \rightleftharpoons NH^*}$",
    "NH* + Haqu <-> NH2*"   : r"$\ce{NH^* + H+ + e- \rightleftharpoons NH2^*}$",
    "NH2* + Haqu <-> NH3*"  : r"$\ce{NH2^* + H+ + e- \rightleftharpoons NH3^*}$",
    "O* + Haqu <-> OH*"     : r"$\ce{O^* + H+ + e- \rightleftharpoons OH^*}$",
    "OH* + Haqu <-> H2O*"   : r"$\ce{OH^* + H+ + e- \rightleftharpoons H2O^*}$",
    "Haqu + * <-> H*"       : r"$\ce{H+ + e- + {}^* \rightleftharpoons H^*}$",
    "H2 + 2 * <-> 2 H*"     : r"$\ce{H2 + 2 {}^* \rightleftharpoons{} 2H^*}$",
}

## 20 Jul 2020: Results

### Parse results

- Define some functions to facilitate parsing the data.

In [ ]:
def parse_mkmcxx_results(
    results_dir: str = ".", U: float = 0.2, n_jobs: int = 16, outfile_path: str = None
):
    """
    Load the results from the specified MKMCXX results folder.

    Read in the MKMCXX results from a results folder created by Jin-Xun Liu's
    code. This will create a pickle file containing a list of objects, each
    containing the `mkmcxx.MicrokineticsSimulation.results()` output for that
    folder.

    Parameters
    ----------
    results_dir : str, optional
        The folder where the results should be located, by default ".". This
        folder should contain subfolders of the form "O_<EO>__N<EN>", where
        -1*<EO> and -1*<EN> are the binding energies of O and N in kj/mol. Each
        subfolder should contain an "output.log" file and a "run" folder
        containing the MKCMXX simulation results.
    U : float, optional
        The applied potential in V vs. RHE, by default 0.2 V.
    n_jobs : int, optional
        The number of multiprocessing jobs to use when parsing the results, by
        default 16.
    outfile_path : str, optional
        The path in which to save the output pickle file, by default
        "jxl-<potential>V-results_<date>.pckl"
    """

    if not outfile_path:
        outfile_path = f"jxl-{U}V-results_{datetime.date.today().strftime('%d-%b-%Y').lower()}.pckl"

    # Verify that results folder exists
    if not os.path.exists(results_dir):
        raise RuntimeError(f"Folder {results_dir} does not exist.")

    # Get list of all directories where MKMCXX was run
    print("Searching directories...")
    data_dirs = glob.glob(os.path.join(results_dir, "*", "output.log"))
    print(f"Found {len(data_dirs)} data directories.")

    # Make simulation object for each folder
    completed_simulations = [
        make_mkmcxx_simulation(folder) for folder in tqdm.tqdm(data_dirs)
    ]

    # Read the results. Parallelize over many operations
    with multiprocessing.Pool(processes=16) as p:
        parsed_results = list(
            tqdm.tqdm(
                p.imap(read_results, completed_simulations),
                total=len(completed_simulations),
            )
        )

    # Pickle to a results file
    print(f"Saving results to pickle file {outfile_path}...")
    save_to_file(
        parsed_results, outfile_path,
    )
    print("Saving done.")

    # Find directories where there were no results available
    dud_directories = [
        sim["directory"]
        for sim in tqdm.tqdm(
            filter(lambda sim: not sim["results"]["range_results"], parsed_results)
        )
    ]

    print(f"{len(dud_directories)} directories unexpectedly had no simulation results.")
    print(dud_directories)

    return True

In [ ]:
def make_mkmcxx_simulation(folder_path: str):
    """
    Quick wrapper to make a `MicrokineticsSimulation` object
    representing a completed simulation directory.
    """

    # Define some constants
    ev_to_kjmol = 96.485  # kJ/mol per eV
    binding_energy_regex = re.compile(r".*O_(?P<EO>\d+)__N_(?P<EN>\d+).*")

    # Load dummy data to enable instantiation of dummy MKMCXX runs
    fake_reactions = load_from_file(os.path.expanduser("base-reaction-set.pckl"))

    # Set up runs
    fake_runs = [{"temp": 300, "time": 1e8, "abstol": 1e-8, "reltol": 1e-8}]

    # Enter some settings from the input.mkm file
    fake_settings = {
        "type": "sequencerun",
        "usetimestamp": False,
        "drc": 0,
        "reagents": ["NO3-", "Haqu"],
        "keycomponents": ["N2", "NO3-", "N2O"],
        "makeplots": 0,
    }

    # Get O and N binding energies
    result = {"U": U}
    try:
        match = binding_energy_regex.match(folder_path)
        result.update(
            {
                label: -float(energy) / ev_to_kjmol
                for label, energy in match.groupdict().items()
            }
        )
    except AttributeError:
        print("Invalid folder name. Must be of format .*O_<EO>__N_<EN>.*")
        raise RuntimeError

    # Instantiate simulation object and add to dictionary
    folder_name = os.path.dirname(folder_path)
    result.update({"directory": folder_name})
    sim = mkmcxx.MicrokineticSimulation(
        reactions=list(fake_reactions.values()),
        settings=fake_settings,
        runs=fake_runs,
        directory=folder_name,
        # run_directory=".",
        run_directory="run",
    )
    result.update({"sim": sim})

    return result

In [ ]:
def read_results(sim_object):
    """
    Quick wrapper function to read simulation results. Want to package
    into a format that can be read by my Jupyter notebook.
    """

    results = {}

    # Copy the binding energies and potential
    results.update(
        {label: sim_object[label] for label in ["EO", "EN", "U", "directory"]}
    )

    # Read the simulation results, and add it to the dictionary, then
    # return the dictionary.
    results.update({"results": sim_object["sim"].read_results()})

    return results

- Now parse the data for 0.1 V and 0.2 V.

In [ ]:
# Now for 0.1 V
results_dir = "0.1V-vs-RHE"
U = 0.1
outfile_path = (
    f"jxl-{U}V-results-h2o-corrected_"
    f"{datetime.date.today().strftime('%d-%b-%Y').lower()}.pckl"
)
result = parse_mkmcxx_results(results_dir=results_dir, U=U, outfile_path=outfile_path)

### Assemble data for plotting

- Need to distill down the results to have just the most important things (DRC, TOF, selectivities).

In [ ]:
def get_rate_points(results: list, compound: str = "NO3-"):
    """
    Read rate information from the entire dataset, handling cases where the
    data does not exist

    Parameters
    ----------
    results : list
        The output of the `read_results` functions defined above.
    compound : str
        The key used to look up rate information in the
        `mkmcxx.MicrokineticsSimulation.get_results()
        ["range_results"]["derivatives"]` object

    Returns
    -------
    list
        A list of the form [{"EO": <EO>, "EN": <EN>, "ratelog": <ratelog>}, ...],
        where <EO> is the O binding energy in eV, <EN> is the N binding energy
        in eV, and <ratelog> is the base-10 logarithm of the formation/consumption
        rate of the compound specified in `compound_str`.
    """

    extracted_rate_points = []

    for result in tqdm.tqdm(results):
        try:
            # Try rounding rates to 6 sig figs.
            raw_rate = result["results"]["range_results"]["derivatives"][compound][0]
            rounded_rate = np.float(f"{raw_rate:.6e}")
            projection = {
                "EO": result["EO"],
                "EN": result["EN"],
                "ratelog": np.log10(-(rounded_rate)),
            }
        except (KeyError, IndexError):
            projection = None

        extracted_rate_points.append(projection)

    # Filter out any None values
    extracted_rate_points = list(filter(lambda x: x, extracted_rate_points))

    return extracted_rate_points

In [ ]:
def get_drc_points(results: list, rxn: str):
    """
    Read rate information from the entire dataset, handling cases where the
    data does not exist

    Parameters
    ----------
    results : list
        The output of the `read_results` functions defined above.
    rxn : str
        The key used to look up rate information in the
        `mkmcxx.MicrokineticsSimulation.get_results()["drc_results"]["drc"]`
        object

    Returns
    -------
    list
        A list of the form [{"EO": <EO>, "EN": <EN>, "drc": <drc>}, ...], where
        <EO> is the O binding energy in eV, <EN> is the N binding energy in eV,
        and <drc> is the Campbell degree-of-rate-control coefficient for the
        reaction specified in `rxn`.
    """

    extracted_drc_points = []

    for result in tqdm.tqdm(results):
        try:
            projection = {
                "EO": result["EO"],
                "EN": result["EN"],
                "drc": np.float(result["results"]["drc_results"]["drc"].loc[rxn, 0]),
            }
        except (KeyError, IndexError):
            projection = None

        extracted_drc_points.append(projection)

    # Filter out any None values
    extracted_drc_points = list(filter(lambda x: x, extracted_drc_points))

    return extracted_drc_points

In [ ]:
# Going to store condensed plot data
condensed_plot_data = {}

In [ ]:
# Get results for 0.1 V. Change potential/file names if you ran simulations
# at a different applied potential.
potentials = [0.1]
datestr = datetime.date.today().strftime('%d-%b-%Y').lower()

for potential in potentials:
    results = load_from_file(f"jxl-{potential}V-results-h2o-corrected_{datestr}.pckl")

    condensed_plot_data.update(
        {
            f"{potential}V": {
                "ratelog": {
                    compound_str: get_rate_points(
                        results=results, compound=compound_str
                    )
                    for compound_str in compound_strings
                },
                "drc": {
                    rxn_string: get_drc_points(results=results, rxn=rxn_string)
                    for rxn_string in rxn_strings
                },
            }
        }
    )

    print(f"Got all data for {potential}V vs. RHE")

In [ ]:
# Save all results
datestr = datetime.date.today().strftime("%d %b %Y").lower().replace(" ", "-")
save_to_file(condensed_plot_data, f"all-condensed-plot-data-h2o-corrected_{datestr}.pckl")

### Assemble a DRC contour plot

- This will be for nitrate-to-nitrate dissociation, the proposed rate-limiting step.

In [ ]:
# Load condensed plot data
condensed_plot_data = load_from_file(
    "all-condensed-plot-data-h2o-corrected_24-jul-2020.pckl"
)

In [ ]:
# Also want to load the marker data set in order to plot the points on the volcano
marker_data_set = load_from_file("latest-volcano-marker-data.pckl")

In [ ]:
# Reindex to plot Ru(211) before Rh(211)
new_idx = [
    "comp-1",
    "comp-1.5",
    "comp-2",
    "comp-3",
    "comp-4",
    "comp-5",
    "comp-6.5",
    "comp-7.5",
    "comp-9",
    "ru211",
    "rh211",
    "pt3ru211"
]

marker_data_set = marker_data_set.loc[new_idx, :]

In [ ]:
def plot_marker_set(row, ax):
    """Apply function to plot data points on plot"""

    ax.plot(
        row["EO"],
        row["EN"],
        color=row["color"],
        marker=row["marker"],
        markersize=10,
        markeredgewidth=row["markeredgewidth"] + 0.7,
        markeredgecolor=row["markeredgecolor"],
        label=row["label"],
        linestyle="None"
    )

In [ ]:
def make_drc_plot(
    data: list,
    rxn_str: str,
    ax: plt.Axes = None,
    outfile_name: str = None,
    clip_threshold: float = 2.0,
    contour_levels: np.ndarray = np.r_[-2:2.1:0.1],
):
    """Make a degree of rate control plot for the specified reaction.

    Parameters
    ----------
    data :
        List containing data of the form [{"EO": <EO>, "EN": <EN>, "drc":
        <drc>}, ...], where <EO> is the O binding energy in eV, <EN> is the N
        binding energy in eV, and <drc> is the Campbell degree of rate control
        coefficient.
    rxn_str : str
        The reaction to be written above the plot, in LaTeX format.
    ax : plt.Axes
        The Axes object on which to place this plot. If none, Figure and Axes
        objects will be created for you.
    outfile_name : str, optional
        Name of graphics file (including extension) to which to export the
        plotted graph, by default None. If None, no plot will be written to
        disk. Passed directly to `matplotlib.pyplot.Figure.savefig`. Has no
        effect if `ax` is specified.
    clip_threshold : float, optional
        Absolute value of the symmetric threshold around 0.0 to which to clip
        DRC data that exceeds that threshold, by default 2.0. For example, if
        the DRC threshold is set to 2.0, all DRC values greater than 2 or less
        than -2 will be set to +2 or -2, respectively.
    contour_levels : ndarray, int
        Array of level values to use when drawing the contours on the DRC
        contour plot. Passed directly to `matplotlib.pyplot.tricontour` and
        `matplotlib.pyplot.tricontourf`
    """

    # Make data frame full of data
    df = pd.DataFrame.from_dict(data=data)

    # Filter out NaN/inf values
    df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

    # Report number of points used for DRC
    n_points = len(df)
    print(f"Found {n_points} non-NaN/inf points to plot.")

    # Clip values outside of DRC \in [-2,2]
    mask = df[df["drc"] > clip_threshold].index
    df.loc[mask, "drc"] = clip_threshold

    mask = df[df["drc"] < -clip_threshold].index
    df.loc[mask, "drc"] = -clip_threshold

    # Reorder columns if necessary
    df = df.loc[:, ["EO", "EN", "drc"]]

    # Extract values
    values = [np.ravel(k) for k in np.hsplit(df.values, 3)]

    # Create DRC plot
    fig = None
    if not ax:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6.5))
        print("Making my own plot")
    contourset = ax.tricontourf(*values, levels=contour_levels, cmap="jet")

    # Plot the marker data
    marker_data_set.apply(func=lambda row: plot_marker_set(row, ax), axis=1)

    # Set x and y ticks to be the same
    ticks = np.r_[-6:-1:1]
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.axis([-6.7,-2.5,-7,-2])

    # Place reaction inside frame
    ax.annotate(
        xy=(0.5, 0.90),
        s=rxn_str,
        xycoords="axes fraction",
        fontsize="medium",
        ha="center",
        va="top",
    )
    # ax.set_xlabel("O binding energy / eV")
    # ax.set_ylabel("N binding energy / eV")
    # ax.axis("square")

    # If user didn't specify axis, no figure will have been created. In this
    # case, add our own color bar and return the figure and axes objects.
    if fig:
        print("Adding my own colorbar")
        fig.colorbar(
            contourset,
            label="Degree of rate control factor",
        )
        if outfile_name:
            fig.savefig(outfile_name)

        return fig, ax
    else:
        # Return the contour set object so it can be used for a figure bar in
        # the larger figure
        return contourset

In [ ]:
# Draw all DRC plots
fig, ax = plt.subplots(
    nrows=5,
    ncols=4,
    figsize=(25, 25),
    # constrained_layout=True,
    sharex=True,
    sharey=True,
)

fig.subplots_adjust(wspace=0.0, hspace=0.0, left=0.135, bottom=0.135)

datestr = datetime.date.today().strftime("%d %b %Y").lower().replace(" ", "-")

plot_results = []
for ax_instance, rxn_string in zip(ax.flat, rxn_strings):
    plot_results.append(
        make_drc_plot(
            data=condensed_plot_data["0.1V"]["drc"][rxn_string],
            rxn_str=rxn_labels_latex[rxn_string],
            ax=ax_instance,
        )
    )
    print(f"Finished plot for {rxn_string}")

# Remove final blank subplot
ax_blank = list(ax.flat)[-1]
ax_blank.set_axis_off()

# Set up common x and y labels
# Thanks to https://stackoverflow.com/a/26892326
fig.text(x=0.45, y=0.1, s="O binding energy / eV", ha="center", fontsize="large")
fig.text(
    x=0.1,
    y=0.5,
    s="N binding energy / eV",
    ha="center",
    rotation="vertical",
    fontsize="large",
)

# Set up color bar
cbar = fig.colorbar(
    mappable=plot_results[-1],
    cmap="jet",
    ax=ax,
    location="right",
    aspect=40,
    shrink=0.70,
    pad=0.02,
)
cbar.set_label(label="Degree of rate control", labelpad=15)

# Add legend in the place of the very last subplot
ax_lastplot = list(ax.flat)[-2]

lg = ax_lastplot.legend(
    loc="right",
    fontsize=18,
    frameon=True,
    handletextpad=-0.10,
    borderpad=0.25,
    labelspacing=0.55,
    facecolor="k",
    framealpha=0.15,
    edgecolor="k",
    bbox_to_anchor=(1.56, 0, 0.5, 1.0),
    ncol=2,
)

# Export graphics
fig.savefig(
    f"no3-her-drc-plots_latest.pdf", dpi=150, bbox_inches="tight", pad_inches=0.1
)

fig.savefig(
    f"no3-her-drc-plots_latest.png", dpi=150, bbox_inches="tight", pad_inches=0.1
)

### Assemble a TOF contour plot

In [ ]:
def make_tof_plot(
    data: list,
    compound_str: str,
    ax: plt.Axes = None,
    outfile_name: str = None,
    contour_levels: np.ndarray = np.r_[-36:3.1:3],
):
    """Make a degree of rate control plot for the specified reaction.

    Parameters
    ----------
    data :
        List containing data of the form [{"EO": <EO>, "EN": <EN>, "rate":
        <rate>}, ...], where <EO> is the O binding energy in eV, <EN> is the N
        binding energy in eV, and <rate> is the Campbell degree of rate control
        coefficient.
    compound_str : str
        The name of the compound (in LaTeX syntax) to show in the title of the
        plot.
    ax : plt.Axes
        The Axes object on which to place this plot. If none, Figure and Axes
        objects will be created for you.
    outfile_name : str, optional
        Name of graphics file (including extension) to which to export the
        plotted graph, by default None. If None, no plot will be written to
        disk. Passed directly to `matplotlib.pyplot.Figure.savefig`. Has no
        effect if `ax` is specified.
    contour_levels : ndarray, int
        Array of level values to use when drawing the contours on the DRC
        contour plot. Passed directly to `matplotlib.pyplot.tricontour` and
        `matplotlib.pyplot.tricontourf`
    """

    # Make data frame full of data
    df = pd.DataFrame.from_dict(data=data)

    # Filter out NaN/inf values
    df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

    # Report number of points used for DRC
    n_points = len(df)
    print(f"Found {n_points} non-NaN/inf points to plot.")

    # Reorder columns if necessary
    df = df.loc[:, ["EO", "EN", "ratelog"]]

    # Extract values
    values = [np.ravel(k) for k in np.hsplit(df.values, 3)]

    # Create TOF plot
    fig = None
    if not ax:
        fig, ax = plt.subplots(1, 1, figsize=(10, 6.5))
        print("Making my own plot")
    contourset = ax.tricontourf(
        *values, levels=contour_levels, cmap="Spectral_r")

    ax.set_title(
        fr"""Volcano plot: {compound_str} TOF""")
    ax.set_xlabel("O binding energy / eV")
    ax.set_ylabel("N binding energy / eV")

    # If user didn't specify axis, no figure will have been created. In this
    # case, add our own color bar and return the figure and axes objects.
    if fig:
        print("Adding my own colorbar")
        fig.colorbar(
            contourset,
            label=r"$\log_{10}(\mathrm{TOF})$ /  $\mathrm{s^{-1}}$",
        )
        if outfile_name:
            fig.savefig(outfile_name)

        return fig, ax
    else:
        # Return the contour set object so it can be used for a figure bar in
        # the larger figure
        return contourset


In [ ]:
# Show TOF volcano plot for nitrate consumption
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

datestr = datetime.date.today().strftime("%d %b %Y").lower().replace(" ", "-")

# Plot DRC for NO3 dissociation and HER
result = make_tof_plot(
    data=condensed_plot_data["0.1V"]["ratelog"]["NO3-"],
    compound_str=r"$\mathrm{NO_3^-}$",
    ax=ax,
)

ax.axis([-6.5,-3.5,-7,-3.5])


cbar = fig.colorbar(
    mappable=result, cmap="Spectral_r", ax=ax, aspect=20
)
cbar.set_label(label=r"$\log_{10}(\mathrm{TOF})$ /  $\mathrm{s^{-1}}$", labelpad=15)

fig.savefig(f"no3-her-ratelog-plots_{datestr}.pdf")